# Simple Agent with Microsoft Agent Framework

This notebook demonstrates a simple agent using the Microsoft Agent Framework with Microsoft Foundry.

The code:

- Creates an agent using `AzureOpenAIResponsesClient` connected to Microsoft Foundry
- Defines a `get_company_info` tool with hardcoded company data
- Shows how the agent decides when to call the tool
- Streams the agent's response

Review and run the code and answer the following questions: 

1. What is the agent's function? 
2. How does the agent decide when to use the tool?
3. What are the limitations of hardcoded data?

***

Load the environment variables and import the required Python modules.

In [ ]:
import sys
sys.path.insert(0, '../shared')

from typing import Annotated

from agent_framework.azure import AzureOpenAIResponsesClient
from azure.identity import AzureCliCredential
from pydantic import Field

from config import get_agent_config

Get the agent configuration from the environment.

In [ ]:
# Get configuration from environment
config = get_agent_config()

## Create the Tools

In MAF, **tools** are simple Python functions. The agent reads the function's **name**, **docstring**, and **parameter annotations** to decide when and how to call it.

```
User asks a question
    → Agent reads tool name + docstring
    → Agent decides: "Do I need this tool?"
    → If yes: calls the function, uses the result to answer
    → If no: answers from its own knowledge
```

The `get_company_info` tool below uses a hardcoded dictionary of SEC 10-K filing companies — the same companies you'll work with throughout the workshop.

In [ ]:
# Hardcoded company data from SEC 10-K filings
COMPANIES = {
    "APPLE": {"name": "APPLE INC", "ticker": "AAPL", "sector": "Technology", "cik": "1490054"},
    "MICROSOFT": {"name": "MICROSOFT CORP", "ticker": "MSFT", "sector": "Technology", "cik": "789019"},
    "NVIDIA": {"name": "NVIDIA CORPORATION", "ticker": "NVDA", "sector": "Technology", "cik": "1045810"},
    "AMAZON": {"name": "AMAZON", "ticker": "AMZN", "sector": "Consumer Cyclical", "cik": "1018724"},
    "INTEL": {"name": "INTEL CORP", "ticker": "INTC", "sector": "Technology", "cik": "50863"},
    "PAYPAL": {"name": "PAYPAL", "ticker": "PYPL", "sector": "Financial Services", "cik": "1633917"},
    "MCDONALDS": {"name": "MCDONALDS CORP", "ticker": "MCD", "sector": "Consumer Cyclical", "cik": "63908"},
    "AIG": {"name": "AMERICAN INTL GROUP", "ticker": "AIG", "sector": "Financial Services", "cik": "5272"},
}

def get_company_info(
    company_name: Annotated[str, Field(description="The company name to look up (e.g. 'Apple', 'Microsoft')")]
) -> str:
    """Look up basic information about a company including its ticker symbol, sector, and SEC CIK number."""
    key = company_name.upper().strip()
    # Try exact match first, then partial match
    info = COMPANIES.get(key)
    if not info:
        for k, v in COMPANIES.items():
            if key in k or k in key:
                info = v
                break
    if info:
        return (
            f"Company: {info['name']}\n"
            f"Ticker: {info['ticker']}\n"
            f"Sector: {info['sector']}\n"
            f"SEC CIK: {info['cik']}"
        )
    available = ", ".join(COMPANIES.keys())
    return f"Company '{company_name}' not found. Available companies: {available}"

# Define a list of tools for the agent
tools = [get_company_info]

> The agent will use the tool's name (`get_company_info`) and docstring (`Look up basic information about a company...`) to determine whether it should execute the tool to resolve a user's query.

***

Create the agent using the Microsoft Agent Framework with Microsoft Foundry.

The `AzureOpenAIResponsesClient` connects to your Microsoft Foundry project. The `as_agent()` method creates a local agent with your instructions and tools.

In [ ]:
# Create credential and client
credential = AzureCliCredential()
client = AzureOpenAIResponsesClient(
    project_endpoint=config.project_endpoint,
    deployment_name=config.model_name,
    credential=credential,
)

# Create the agent
agent = client.as_agent(
    name="workshop-company-agent",
    instructions=(
        "You are a helpful assistant that answers questions about companies "
        "in SEC 10-K filings. Use your tool to look up company information "
        "when asked about a specific company."
    ),
    tools=tools,
)

print(f"Agent '{agent.name}' created")

Define a reusable `ask` function to query the agent and stream the response.

When you run a query, you will see:

1. The user's question
2. The agent's response (which may include data retrieved by the tool)

In [ ]:
async def ask(query):
    """Run a query against the agent and stream the response."""
    print(f"User: {query}\n")
    print("Assistant: ", end="", flush=True)
    async for update in agent.run_stream(query):
        if update.text:
            print(update.text, end="", flush=True)
    print("\n")

await ask("What can you tell me about Apple?")

Run the cells below to experiment with different queries. You can also modify the queries or add new cells.

> Notice how the agent calls the tool when asked about a specific company, but answers general questions using its own knowledge.

***

[Continue to the Context Providers Notebook](02_context_provider.ipynb)

In [ ]:
await ask("Compare the sectors of NVIDIA and PayPal")

In [ ]:
await ask("Which companies do you have information about?")

In [ ]:
await ask("What is a 10-K filing?")

In [ ]:
# No cleanup needed — AzureOpenAIResponsesClient and sync credentials
# don't require explicit lifecycle management